# Playground for Topic Modeling slides
- Stephen W. Thomas
- Used for MMA 865; MMAI 891; Exec Ed

In [1]:
import datetime
print(datetime.datetime.now())

2018-12-17 19:28:45.538438


In [2]:
import pandas as pd
import numpy as np

import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns 

import re
import string
import unicodedata

import nltk
from nltk.corpus import stopwords
from nltk import word_tokenize
from nltk import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
from nltk.stem import LancasterStemmer, WordNetLemmatizer

from bs4 import BeautifulSoup

import os

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

# Read in the Data

In [3]:
in_dir = "C:/Users/st50/Documents/sandbox/data"
out_dir = "C:/Users/st50/Documents/sandbox/out"

%time kiva_df=pd.read_csv(os.path.join(in_dir, "kiva.csv"))
#kiva_df = kiva_df.sample(frac=0.1)

kiva_df = kiva_df.dropna()
kiva_df = kiva_df.reset_index(drop=True)
kiva_df['en_length'] = kiva_df['en'].apply(len)


list(kiva_df)
kiva_df.info()
kiva_df.shape
kiva_df.head()
kiva_df.tail()

Wall time: 113 ms


['status',
 'sector',
 'en',
 'country',
 'gender',
 'loan_amount',
 'nonpayment',
 'en_length']

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7882 entries, 0 to 7881
Data columns (total 8 columns):
status         7882 non-null object
sector         7882 non-null object
en             7882 non-null object
country        7882 non-null object
gender         7882 non-null object
loan_amount    7882 non-null float64
nonpayment     7882 non-null object
en_length      7882 non-null int64
dtypes: float64(1), int64(1), object(6)
memory usage: 492.7+ KB


(7882, 8)

,status,sector,en,country,gender,loan_amount,nonpayment,en_length
0,defaulted,Agriculture,<i>This description is written by Rowland Amul...,Kenya,M,500.0,lender,897
1,defaulted,Food,<i>This description is written by Rowland Amul...,Kenya,F,500.0,lender,1279
2,defaulted,Food,<i>This description was written by Richard Maz...,Kenya,M,500.0,lender,897
3,defaulted,Services,<i>This description was written by Rowland Amu...,Kenya,F,500.0,lender,737
4,defaulted,Construction,<i>This description was written by Rowland Amu...,Kenya,M,500.0,lender,798


,status,sector,en,country,gender,loan_amount,nonpayment,en_length
7877,paid,Clothing,Rosa is a member of the 3 de Mayo Communal Ban...,Ecuador,F,1050.0,lender,1401
7878,paid,Agriculture,"The communal bank, ""Cumbe,"" is located on the ...",Ecuador,F,2650.0,partner,5315
7879,paid,Food,Gladys Burgos Macas is a member of the 3 de Ma...,Ecuador,F,775.0,partner,809
7880,paid,Retail,The Communal Bank <em>Las Caleras</em> is loca...,Ecuador,F,275.0,partner,1800
7881,paid,Services,Eliud Ouma owns the Purple Hut salon located o...,Kenya,M,275.0,partner,642


# Text Preprocessing

In [4]:
from nltk.corpus import stopwords 
from bs4 import BeautifulSoup
import unidecode

import warnings

with warnings.catch_warnings():
    warnings.filterwarnings("ignore",category=DeprecationWarning)

stop_words = set(stopwords.words('english') + stopwords.words('spanish'))

lemmer = WordNetLemmatizer()

def preprocess(x):
    # Remove HTML tags
    # (A note about the warnings: BeautifulSoup is throwing lots of DepecationWarnings, and 
    # I just don't want to see them right now.)
    with warnings.catch_warnings():
        warnings.filterwarnings("ignore",category=DeprecationWarning)
        x = BeautifulSoup(x, "lxml").get_text()

    # Lower case
    x = x.lower()
    
    # Remove punctuation
    x = re.sub(r'[^\w\s]', '', x)
    
    # Remove non-unicode
    x = unidecode.unidecode(x)
    
    # Remove numbers
    x = re.sub(r'\d+', '', x)
    
    # Remove stopwords and lemmatize
    x = [lemmer.lemmatize(w) for w in x.split() if w not in stop_words]
    return ' '.join(x) 

%time kiva_df['en_clean'] = kiva_df['en'].apply(preprocess)
kiva_df.head()

Wall time: 6.66 s


,status,sector,en,country,gender,loan_amount,nonpayment,en_length,en_clean
0,defaulted,Agriculture,<i>This description is written by Rowland Amul...,Kenya,M,500.0,lender,897,description written rowland amulyoto staff mem...
1,defaulted,Food,<i>This description is written by Rowland Amul...,Kenya,F,500.0,lender,1279,description written rowland amulyoto staff mem...
2,defaulted,Food,<i>This description was written by Richard Maz...,Kenya,M,500.0,lender,897,description written richard mazengo staff memb...
3,defaulted,Services,<i>This description was written by Rowland Amu...,Kenya,F,500.0,lender,737,description written rowland amulyoto staff mem...
4,defaulted,Construction,<i>This description was written by Rowland Amu...,Kenya,M,500.0,lender,798,description written rowland amulyoto staff mem...


## Topic Modeling with Sci-kit Learn

In [5]:
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer

no_features = 1000

tf_vectorizer = CountVectorizer(max_df=0.5, min_df=0.02, 
                                max_features=no_features, ngram_range=[1,3])
%time dtm_tf = tf_vectorizer.fit_transform(kiva_df['en_clean'])
print(dtm_tf.shape)

tfidf_vectorizer = TfidfVectorizer(max_df=0.5, min_df=0.05, 
                                   max_features=no_features, ngram_range=[1,3])
%time dtm_tfidf = tfidf_vectorizer.fit_transform(kiva_df['en_clean'])
print(dtm_tfidf.shape)

# Choose which of the above we'll use below
dtm = dtm_tf
vectorizer = tf_vectorizer

feature_names = vectorizer.get_feature_names()
def filter_words(x):
    return ' '.join([w for w in x.split() if w in feature_names])

# Create a new column, which is the same as en_clean, but only keeps the 
# words from the vectorizer's vocabulary
%time kiva_df['en_clean_filter'] = kiva_df['en_clean'].apply(filter_words)

Wall time: 4.47 s
(7882, 996)
Wall time: 4.84 s
(7882, 342)
Wall time: 5.2 s


In [6]:
from sklearn.decomposition import LatentDirichletAllocation

def runLDA(dtm, vectorizer, n_components, alpha=None, beta=None):

    lda_model = LatentDirichletAllocation(n_components=n_components,
                                          doc_topic_prior=alpha,
                                          topic_word_prior=beta,
                                          max_iter=200, 
                                          learning_method='batch', 
                                          random_state=123,
                                          n_jobs=2,
                                          verbose=0)
    lda_output = lda_model.fit(dtm)

    # Log Likelyhood: Higher the better
    ll = lda_model.score(dtm)

    # Perplexity: Lower the better. Perplexity = exp(-1. * log-likelihood per word)
    perp = lda_model.perplexity(dtm)

    # Theta = document-topic matrix
    # Beta = components_ = topic-term matrix
    theta = pd.DataFrame(lda_model.transform(dtm))
    beta = pd.DataFrame(lda_model.components_)
    
    # Build Topic Summary
    no_top_words = 10
    weight = theta.sum(axis=0)
    support50 = (theta > 0.5).sum(axis=0)
    support10 = (theta > 0.1).sum(axis=0)
    termss = list()
    for topic_id, topic in enumerate(lda_model.components_):
        terms = " ".join([feature_names[i] for i in topic.argsort()[:-no_top_words - 1:-1]])
        termss.append(terms)
    topic_summary = pd.DataFrame({'TopicID': range(0, len(termss)), "Support50": support50, "Support10": support10, "Weight": weight, "Terms": termss})
    
    return {'model':lda_model, 'theta':theta, 'beta':beta, 'topic_summary':topic_summary, 'll':ll, 'perp':perp}

%time l10 = runLDA(dtm, vectorizer, 10)
%time l15 = runLDA(dtm, vectorizer, 15)
%time l20 = runLDA(dtm, vectorizer, 20)
%time l1022 = runLDA(dtm, vectorizer, 10, alpha=0.2, beta=0.2)
%time l1011 = runLDA(dtm, vectorizer, 10, alpha=0.02, beta=0.02)

Wall time: 6min 57s
Wall time: 7min 7s
Wall time: 7min
Wall time: 8min 47s
Wall time: 7min 22s


# Display the Topic Summaries

In [12]:
pd.set_option('display.max_colwidth', 100)
pd.set_option('display.max_columns', 0)

from IPython.display import Markdown, display

def print_document(document_id, theta, res):
    str_list = []
    text = kiva_df['en_clean_filter'].iloc[document_id]
    Topics = theta.iloc[document_id, :].nlargest(3)
    TopicIDs = Topics.index
    
    display(Markdown("#### Document {} (status={}, country={}, gender={}, loan_amount={}, sector={}, nonpayment={})".format(document_id, 
                                                                                            kiva_df['status'].iloc[document_id], 
                                                                                            kiva_df['country'].iloc[document_id],
                                                                                            kiva_df['gender'].iloc[document_id],
                                                                                            kiva_df['loan_amount'].iloc[document_id],
                                                                                            kiva_df['sector'].iloc[document_id],
                                                                                            kiva_df['nonpayment'].iloc[document_id])))
    display(Markdown("###### Top 3 topics"))
    topic_id = TopicIDs[0]
    topic_str=list()
    display(Markdown('- Topic {} ({:.2f}): <span style="color:green;">{}</span>'.format(topic_id, theta.iloc[document_id, topic_id], res['Terms'].iloc[topic_id])))
    topic_id = TopicIDs[1]
    display(Markdown('- Topic {} ({:.2f}): <span style="color:blue;">{}</span>'.format(topic_id, theta.iloc[document_id, topic_id], res['Terms'].iloc[topic_id])))
    topic_id = TopicIDs[2]
    display(Markdown('- Topic {} ({:.2f}): <span style="color:purple;">{}</span>'.format(topic_id, theta.iloc[document_id, topic_id], res['Terms'].iloc[topic_id])))
    
    topic0_words = res['Terms'].iloc[TopicIDs[0]].split()
    topic1_words = res['Terms'].iloc[TopicIDs[1]].split()
    topic2_words = res['Terms'].iloc[TopicIDs[2]].split()

    for word in text.split():
        if word in topic0_words:
            str_list.append('<span style="color:green;font-weight:bold">' + word + '</span>')
        elif word in topic1_words:
            str_list.append('<span style="color:blue;font-weight:bold">' + word + '</span>')
        elif word in topic2_words:
            str_list.append('<span style="color:purple;font-weight:bold">' + word + '</span>')
        else:
            str_list.append('<span style="color:grey">' + word + '</span>')

    display(Markdown("###### Words (After Preprocessing and Filtering) - Highlighted"))
    display(Markdown(' '.join(str_list)))
    
    display(Markdown("###### Words (Original)"))
    display((kiva_df['en'].iloc[document_id]))
    
    
def show_topics(run):
    
    params = run['model'].get_params()
    display(Markdown("# Run: K={}, Alpha={}, Beta={}, Iters={}".format(
        params['n_components'], 
        params['doc_topic_prior'],
        params['topic_word_prior'],
        params['max_iter'],
    )))
    
    display(run['topic_summary'])
    
    for topic_id in range(run['topic_summary'].shape[0]):
        display(Markdown("## Topic {}".format(topic_id)))
        Memberships = run['theta'].iloc[:, topic_id].nlargest(5)
        DocIDs = Memberships.index
        for doc_id in DocIDs:
            print_document(doc_id, run['theta'], run['topic_summary'])

show_topics(l10)
show_topics(l15)
show_topics(l20)
show_topics(l1022)
show_topics(l1011)

# Run: K=10, Alpha=None, Beta=None, Iters=200

,TopicID,Support50,Support10,Weight,Terms
0,0,252,2286,697.833700,buy husband food small work two run chicken mother old
1,1,200,204,170.141876,many risk lender kenya due additional remains entrepreneur recent community
2,2,597,618,535.491856,client mifex ecuador offer service program access offer client visit educational
3,3,35,220,105.589139,water rice month also must farmer high rural sector many
4,4,1089,1790,1096.768540,work old volunteer spanish kiva kiva volunteer year old home help translated
5,5,120,316,186.732845,rice farmer land sector area also crop part people bank
6,6,957,1674,999.855802,group member usd school woman dairy cow married milk woman group
7,7,280,1739,575.709999,first second product kiva first loan second loan able new also home
8,8,1798,2527,1675.836623,group community woman small hope clothing life family one entrepreneur
9,9,1815,3134,1838.039621,school family able stock married two buy also expand purchase


## Topic 0

#### Document 3627 (status=paid, country=Ecuador, gender=M, loan_amount=475.0, sector=Food, nonpayment=partner)

###### Top 3 topics

...

- Topic 1 (0.00): <span style="color:purple;">many risk lender kenya due additional entrepreneur recent community local</span>

###### Words (After Preprocessing and Filtering) - Highlighted

<span style="color:grey">old</span> <span style="color:green;font-weight:bold">married</span> <span style="color:grey">woman</span> <span style="color:grey">attend</span> <span style="color:green;font-weight:bold">school</span> <span style="color:grey">support</span> <span style="color:blue;font-weight:bold">husband</span> <span style="color:green;font-weight:bold">family</span> <span style="color:grey">need</span> <span style="color:grey">running</span> <span style="color:grey">second</span> <span style="color:grey">hand</span> <span style="color:grey">clothes</span> <span style="color:grey">open</span> <span style="color:grey">market</span> <span style="color:grey">within</span> <span style="color:grey">town</span> <span style="color:grey">centre</span> <span style="color:grey">cover</span> <span style="color:grey">last</span> <span style="color:grey">ten</span> <span style="color:grey">good</span> <span style="color:grey">customer</span> <span style="color:grey">base</span> <span style="color:grey">result</span> <span style="color:grey">successful</span> <span style="color:green;font-weight:bold">able</span> <span style="color:grey">assist</span> <span style="color:grey">childrens</span> <span style="color:grey">cost</span> <span style="color:green;font-weight:bold">family</span> <span style="color:grey">need</span> <span style="color:grey">dream</span> <span style="color:grey">see</span> <span style="color:grey">get</span> <span style="color:grey">college</span> <span style="color:grey">education</span> <span style="color:grey">since</span> <span style="color:grey">unable</span> <span style="color:grey">primary</span> <span style="color:grey">level</span> <span style="color:grey">education</span> <span style="color:purple;font-weight:bold">due</span> <span style="color:grey">lack</span> <span style="color:green;font-weight:bold">school</span> <span style="color:grey">fee</span> <span style="color:grey">get</span> <span style="color:grey">clothes</span> <span style="color:purple;font-weight:bold">local</span> <span style="color:grey">second</span> <span style="color:grey">hand</span> <span style="color:grey">result</span> <span style="color:grey">high</span> <span style="color:grey">cost</span> <span style="color:grey">one</span> <span style="color:grey">clothes</span> <span style="color:grey">income</span> <span style="color:grey">little</span> <span style="color:grey">support</span> <span style="color:green;font-weight:bold">family</span> <span style="color:grey">time</span> <span style="color:grey">allow</span> <span style="color:green;font-weight:bold">expand</span> <span style="color:grey">would</span> <span style="color:grey">like</span> <span style="color:grey">increase</span> <span style="color:green;font-weight:bold">stock</span> <span style="color:grey">earn</span> <span style="color:grey">profit</span> <span style="color:grey">cannot</span> <span style="color:grey">meet</span> <span style="color:grey">plan</span> <span style="color:purple;font-weight:bold">due</span> <span style="color:grey">capital</span> <span style="color:grey">requesting</span> <span style="color:grey">help</span> <span style="color:grey">meet</span> <span style="color:grey">plan</span> <span style="color:grey">fund</span> <span style="color:grey">used</span> <span style="color:green;font-weight:bold">buy</span> <span style="color:green;font-weight:bold">two</span> <span style="color:grey">clothes</span> <span style="color:grey">hardworking</span> <span style="color:grey">woman</span> <span style="color:green;font-weight:bold">able</span> <span style="color:grey">repay</span> <span style="color:grey">good</span> <span style="color:grey">time</span>

###### Words (Original)

'<p>Lucy Muthoni Njenga is a 42 year old married woman with 3 children. All her children attend school. She supports her husband in meeting the family needs by running a second hand clothes business in an open market within the town centre. She sells an assortment of mattress covers and bed sheets. She has been doing this business for the last ten years and has cultivated a good customer base. As a result of her successful business she has been able to assist in meeting her childrens academic cost and other family needs. Her dream is to see her children get a college education since she was unable to complete her primary level education due to lack of school fees. \r\n<p>She gets her bale of clothes from a local second hand broker. Byuing from an intermediary, results in a high cost of one bale of clothes. Her income is too little to support her family and at the same time allow her to expand her business. She would like to increase her stock to earn more profit but she cannot meet her

# Visualize Topics with LDAVis

In [8]:
import pyLDAvis.sklearn
 
pyLDAvis.enable_notebook()
% time pyLDAvis.sklearn.prepare(l15['model'], dtm, vectorizer, mds="tsne")

C:\Users\st50\AppData\Local\Continuum\anaconda3\lib\site-packages\pyLDAvis\_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


Wall time: 18.9 s


PreparedData(topic_coordinates=                x           y  topics  cluster       Freq
topic                                                    
9      719.432312  285.289581  1       1        16.721390
8     -117.047333 -25.829000   2       1        13.783107
4     -654.934937  456.201965  3       1        9.761625 
2      34.486206   409.543427  4       1        8.487657 
12     196.668396 -183.041061  5       1        8.084441 
6     -285.525482  704.799438  6       1        8.070031 
0     -303.678040  271.458466  7       1        7.629120 
5      435.209991  575.453857  8       1        6.056585 
7      298.377350  168.846664  9       1        5.239373 
1     -28.811169  -531.733215  10      1        3.859697 
14    -386.522247 -337.000305  11      1        3.085741 
11    -619.630676 -2.256888    12      1        2.770535 
3      118.781456  831.262268  13      1        2.486532 
13     623.627869 -118.996208  14      1        2.465671 
10     428.402985 -508.414459  15      1        1.498495 , topic_info=     Category         Freq   ...     loglift  logprob
term                         ...                     
362   Default  5057.000000   ...     30.0000  30.0000
757   Default  2672.000000   ...     29.0000  29.0000
304   Default  2386.000000   ...     28.0000  28.0000
554   Default  3252.000000   ...     27.0000  27.0000
811   Default  1924.000000   ...     26.0000  26.0000
167   Default  3215.000000   ...     25.0000  25.0000
202   Default  1071.000000   ...     24.0000  24.0000
199   Default  2435.000000   ...     23.0000  23.0000
190   Default  1034.000000   ...     22.0000  22.0000
571   Default  1042.000000   ...     21.0000  21.0000
933   Default  1345.000000   ...     20.0000  20.0000
319   Default  1989.000000   ...     19.0000  19.0000
453   Default  1679.000000   ...     18.0000  18.0000
792   Default  1078.000000   ...     17.0000  17.0000
154   Default  2506.000000   ...     16.0000  16.0000
442   Default  2116.000000   ...     15.0000  15.0000
952   Default  1108.000000   ...     14.0000  14.0000
970   Default  3171.000000   ...     13.0000  13.0000
974   Default  3945.000000   ...     12.0000  12.0000
690   Default  3721.000000   ...     11.0000  11.0000
273   Default  1612.000000   ...     10.0000  10.0000
797   Default  1265.000000   ...     9.0000   9.0000 
320   Default  942.000000    ...     8.0000   8.0000 
305   Default  1068.000000   ...     7.0000   7.0000 
568   Default  1672.000000   ...     6.0000   6.0000 
779   Default  742.000000    ...     5.0000   5.0000 
52    Default  2242.000000   ...     4.0000   4.0000 
867   Default  2608.000000   ...     3.0000   3.0000 
39    Default  5173.000000   ...     2.0000   2.0000 
70    Default  1254.000000   ...     1.0000   1.0000 
...       ...          ...   ...        ...      ... 
212   Topic15  56.829961     ...     2.9011  -5.0124 
535   Topic15  56.829961     ...     2.9011  -5.0124 
533   Topic15  56.829961     ...     2.9011  -5.0124 
727   Topic15  56.829961     ...     2.9011  -5.0124 
763   Topic15  56.829961     ...     2.9011  -5.0124 
962   Topic15  56.829961     ...     2.9011  -5.0124 
550   Topic15  56.829961     ...     2.9011  -5.0124 
549   Topic15  56.829961     ...     2.9011  -5.0124 
760   Topic15  123.785657    ...     2.4073  -4.2339 
741   Topic15  59.198378     ...     2.8461  -4.9716 
531   Topic15  135.164894    ...     1.5159  -4.1460 
8     Topic15  56.829961     ...     2.9011  -5.0124 
282   Topic15  60.229639     ...     2.6868  -4.9543 
238   Topic15  77.842280     ...     1.4265  -4.6978 
273   Topic15  80.253084     ...     1.2006  -4.6673 
788   Topic15  95.733436     ...     0.2222  -4.4909 
7     Topic15  56.829961     ...     2.9011  -5.0124 
798   Topic15  61.202216     ...     2.2821  -4.9383 
828   Topic15  60.615154     ...     2.3306  -4.9480 
486   Topic15  57.950004     ...     2.7171  -4.9929 
27    Topic15  56.829961     ...     2.9011  -5.0124 
527   Topic15  63.859679     ...     1.4614  -4.8958 
4